In [197]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

conn = sqlite3.connect('/home/kub/projects/ML/Bezrealitky.db')
df = pd.read_sql_query("SELECT * FROM tab_parametry", conn)
df

,Cislo_inzeratu,Dispozice,Stav,Plocha,Cena,Mesto,Mestska,Vlastnictvi,Typ,PENB,...,Sklep,Lodzie,Parkovani,Vytah,Garaz,K,Rekonstrukce,Odkaz,Datum_pridani,Datum_odebrani
0,677693,2+kk,Velmi dobrý,38 m²,19.000.000 Kč,Praha,Staré Město,Osobní,Ostatní,A,...,Ne,Ano,Ne,Ano,Ne,15. 09. 2021,vše,/home/kub/projects/bezrealitky/fotky/1,15:05:51,
1,691463,2+1,Špatný,63 m²,3.800.000 Kč,Liberec,-,Osobní,Panel,-,...,Ano,Ne,Ne,Ano,Ne,-,-,/home/kub/projects/bezrealitky/fotky/2,15:05:51,
2,691460,1+1,Dobrý,27 m²,2.521.000 Kč,Plzeň,Doubravka,Osobní,Cihla,-,...,Ano,Ne,Ne,Ne,Ne,-,-,/home/kub/projects/bezrealitky/fotky/3,15:05:51,
3,649809,2+1,Velmi dobrý,65 m²,2.290.000 Kč,Ústí nad Labem,-,Osobní,Panel,D,...,Ne,Ano,Ne,Ne,Ne,07. 02. 2021,-,/home/kub/projects/bezrealitky/fotky/4,15:05:51,
4,681552,4+kk,Ve výstavbě,112 m²,5.999.000 Kč,Praha,Hlubočepy,Družstevní,Ostatní,B,...,Ano,Ano,Ne,Ano,Ano,01. 06. 2023,-,/home/kub/projects/bezrealitky/fotky/5,15:05:51,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,418519,3+1,Dobrý,77 m²,2.695.000 Kč,Jeseník,-,Osobní,Panel,D,...,Ano,Ne,Ne,Ne,Ne,01. 01. 1970,exteriér,/home/kub/projects/bezrealitky/fotky/799,15:07:33,
799,632086,3+kk,Velmi dobrý,73 m²,7.550.000 Kč,Praha,Krč,Osobní,Cihla,E,...,Ano,Ne,Ano,Ne,Ne,01. 09. 2021,interiér,/home/kub/projects/bezrealitky/fotky/6,15:24:49,
800,691472,3+kk,Novostavba,91 m²,13.980.000 Kč,Praha,Bubeneč,Osobní,Cihla,D,...,Ano,Ne,Ne,Ano,Ne,01. 01. 2022,-,/home/kub/projects/bezrealitky/fotky/9,15:24:49,
801,690771,2+1,Dobrý,59 m²,6.500.000 Kč,Praha,Chodov,Osobní,Panel,D,...,Ano,Ano,Ne,Ano,Ne,15. 11. 2021,-,/home/kub/projects/bezrealitky/fotky/10,15:24:49,


In [175]:
"""
V modelu bude:
cena = 
dispozice



stav

plocha - v m2

Praha (ano/ne) - 1/0

vlastnictvi - 1 osobní, 0 družstevní


vybavenost

PENB - G-1, F-2, E-3, D-4, C-5, B-6, A-7

balkon - 1/0
terasa- 1/0
sklep - 1/0
lodzie - 1/0
parkovani - 1/0
terasa - 1/0
vytah - 1/0
garaz - 1/0
"""
#set(df["Mesto"].values.tolist())
mesta = {'Aš':13105,
 'Beroun':19496,
 'Besednice':831,
 'Borotín':407,
 'Brandýs nad Orlicí':1377,
 'Brno':379526,
 'Cerhenice':1614,
 'Cheb':32071,
 'Chlumec':4476,
 'Chomutov':487013,
 'Chotětov':848,
 'Chrudim':23083,
 'Chvaletice':2933,
 'Chýně':2006,
 'Doksy':5191,
 'Děčín':49005,
 'Františkovy Lázně':5481,
 'Frenštát pod Radhoštěm':10845,
 'Frýdek-Místek':56066,
 'Havířov':72146,
 'Heřmanův Městec':4997,
 'Holubice':1659,
 'Horní Slavkov':5431,
 'Horušice':205,
 'Hostivice':8224,
 'Hořovice':6790,
 'Hradec Králové':92763,
 'Hradištko':1859,
 'Hranice':18092,
 'Hulín':6834,
 'Hustopeče':5746,
 'Jablonec nad Nisou':45765,
 'Janov nad Nisou':1377,
 'Jaroměř':12439,
 'Jesenice':13255,
 'Jeseník':11204,
 'Jihlava':50891,
 'Jinočany':1419,
 'Jičín':16551,
 'Kadaň':18111,
 'Kamenice':4233,
 'Karlovy Vary':48723,
 'Karviná':53209,
 'Kařez':591,
 'Kladno':68971,
 'Klecany':3142,
 'Klášterec nad Ohří':14538,
 'Kolín':31550,
 'Kopřivnice':22044,
 'Kostelec nad Labem':3616,
 'Kořenov':956,
 'Krupka':12666,
 'Kutná Hora':20622,
 'Kuřim':11008,
 'Libavské Údolí':590,
 'Liberec':104163,
 'Litvínov':23908,
 'Loket':2978,
 'Louny':18395,
 'Lysá nad Labem':9606,
 'Mariánské Lázně':12897,
 'Mikulášovice':2151,
 'Milešovice':663,
 'Milovice':10140,
 'Mladá Boleslav':44327,
 'Mníšek pod Brdy':4632,
 'Mokrá-Horákov':2761,
 'Moravský Beroun':3004,
 'Most':66401,
 'Měcholupy':918,
 'Mělník':19448,
 'Mšecké Žehrovice':528,
 'Nehvizdy':2166,
 'Neratovice':16174,
 'Nová Paka':9103,
 'Nová Ves pod Pleší':984,
 'Nová Včelnice':2306,
 'Nové Město na Moravě':10089,
 'Nové Strašecí':5479,
 'Nový Bor':11679,
 'Nový Jičín':23565,
 'Olomouc':100408,
 'Opava':56834,
 'Ostrava':289629,
 'Ostrov':16802,
 'Pardubice':90458,
 'Pelhřimov':16124,
 'Plesná':1940,
 'Plzeň':171707,
 'Poděbrady':14192,
 'Praha':1309000,
 'Písek':30279,
 'Předklášteří':1443,
 'Přelouč':9520,
 'Přibyslavice':776,
 'Příbram':32757,
 'Rakovník':15909,
 'Rokytnice nad Jizerou':2877,
 'Roudnice nad Labem':12996,
 'Rovná':381,
 'Rudná':4882,
 'Rychnov nad Kněžnou':11028,
 'Sedlčany':7124,
 'Sezimovo Ústí':7211,
 'Slaný':15737,
 'Smiřice':2808,
 'Smržovka':3541,
 'Sokolov':23334,
 'Studénka':9589,
 'Stříbro':7685,
 'Světlá nad Sázavou':6610,
 'Telč':5313,
 'Teplice':49514,
 'Tečovice':1287,
 'Tišnov':9240,
 'Tlučná':3036,
 'Trmice':3967,
 'Trutnov':30459,
 'Třeboň':8265,
 'Třebíč':35853,
 'Třemošná':4878,
 'Třinec':35224,
 'Uničov':11415,
 'Valašské Meziříčí':22183,
 'Velké Meziříčí':11503,
 'Velké Poříčí':2354,
 'Vimperk':7411,
 'Vlašim':11625,
 'Vrchlabí':12461,
 'Vyšší Brod':2447,
 'Všemina':1107,
 'Všeradice':435,
 'Zbýšov':3728,
 'Zlaté Hory':3762,
 'Zlín':74835,
 'Znojmo':33757,
 'Ústí nad Labem':93020,
 'Ústí nad Orlicí':14197,
 'Úvaly':6112,
 'Čachrov':511,
 'Čelákovice':12255,
 'Česká Lípa':37467,
 'Česká Třebová':15547,
 'České Budějovice':93984,
 'České Velenice':3460,
 'Říčany':15517,
 'Šestajovice':3377,
 'Špindlerův Mlýn':1502,
 'Štěchovice':1741,
 'Žamberk':6093,
 'Žatec':19144,
 'Železný Brod':6057,
 'Žďár nad Sázavou':20934}


In [202]:
#df = pd.read_sql_query("SELECT * FROM tab_parametry", conn)

df2 = df
df2 = df2.drop(columns = ["Cislo_inzeratu", "Mestska", "Podlazi", "Typ", "K", "Rekonstrukce", "Odkaz", "Datum_pridani", "Datum_odebrani"])
#df2

df2.columns = df2.columns.str.replace("Vlastnictvi", "Osobni_vl")
df2.loc[(df2.Osobni_vl == "Osobní"), "Osobni_vl"] = 1 #1 pro osobní vlastnictví, 0 pro družstevní
df2.loc[(df2.Osobni_vl != 1),        "Osobni_vl"] = 0

df2["Populace"] = 1
i = 0
for a in df2["Populace"]:
    df2["Populace"][i] = mesta.get(df.iloc[:,5][i])
    #print(a)
    i=i+1

#df2.columns = df2.columns.str.replace("Mesto", "Praha")
#df2.loc[(df2.Praha == "Praha"), "Praha"] = 1 #1 pro Prahu, 0 pro zbytekí
#df2.loc[(df2.Praha != 1), "Praha"] = 0

df2.loc[(df2.PENB == "G"), "PENB"] = 1 #
df2.loc[(df2.PENB == "F"), "PENB"] = 2
df2.loc[(df2.PENB == "E"), "PENB"] = 3
df2.loc[(df2.PENB == "D"), "PENB"] = 4
df2.loc[(df2.PENB == "C"), "PENB"] = 5
df2.loc[(df2.PENB == "B"), "PENB"] = 6
df2.loc[(df2.PENB == "A"), "PENB"] = 7
df2.loc[(df2.PENB == "-"), "PENB"] = 1

df2.loc[(df2.Vybavenost == "Nevybavený"), "Vybavenost"] = 1
df2.loc[(df2.Vybavenost == "Částečně"), "Vybavenost"] = 2
df2.loc[(df2.Vybavenost == "Vybavený"), "Vybavenost"] = 3
df2.loc[(df2.Vybavenost == "-"), "Vybavenost"] = 1

anone_parametry = ["Balkon", "Terasa", "Sklep", "Lodzie", "Parkovani", "Vytah", "Garaz"]

df2.loc[(df2.Balkon == "Ano"), "Balkon"] = 1
df2.loc[(df2.Balkon == "Ne"),  "Balkon"] = 0
df2.loc[(df2.Balkon == "-"),  "Balkon"] = 0

df2.loc[(df2.Terasa == "Ano"), "Terasa"] = 1
df2.loc[(df2.Terasa == "Ne"),  "Terasa"] = 0
df2.loc[(df2.Terasa == "-"),  "Terasa"] = 0

df2.loc[(df2.Sklep == "Ano"), "Sklep"] = 1
df2.loc[(df2.Sklep == "Ne"),  "Sklep"] = 0
df2.loc[(df2.Sklep == "-"),  "Sklep"] = 0

df2.loc[(df2.Lodzie == "Ano"), "Lodzie"] = 1
df2.loc[(df2.Lodzie == "Ne"),  "Lodzie"] = 0
df2.loc[(df2.Lodzie == "-"),  "Lodzie"] = 0

df2.loc[(df2.Parkovani == "Ano"), "Parkovani"] = 1
df2.loc[(df2.Parkovani == "Ne"),  "Parkovani"] = 0
df2.loc[(df2.Parkovani == "-"),  "Parkovani"] = 0

df2.loc[(df2.Vytah == "Ano"), "Vytah"] = 1
df2.loc[(df2.Vytah == "Ne"),  "Vytah"] = 0
df2.loc[(df2.Vytah == "-"),  "Vytah"] = 0

df2.loc[(df2.Garaz == "Ano"), "Garaz"] = 1
df2.loc[(df2.Garaz == "Ne"),  "Garaz"] = 0
df2.loc[(df2.Garaz == "-"),  "Garaz"] = 0

df2["Plocha"] = df2['Plocha'].str[:-2].astype(int) #zbavim se m2
df2['Cena'] = df2['Cena'].str.replace('.','')
df2["Cena"] = df2['Cena'].str[:-2]#.astype(int)
df2 = df2[df2.Cena != "-"]
df2["Cena"] = pd.to_numeric(df2["Cena"])
df2.dropna(inplace=True)
df2["Cena"] = df2['Cena'].astype(int)

df2.loc[(df2.Stav == "Špatný"), "Stav"] = 1
df2.loc[(df2.Stav == "Dobrý"), "Stav"] = 2
df2.loc[(df2.Stav == "Velmi dobrý"), "Stav"] = 3
df2.loc[(df2.Stav == "Ve výstavbě"), "Stav"] = 4
df2.loc[(df2.Stav == "Novostavba"), "Stav"] = 5
df2.loc[(df2.Stav == "-"), "Stav"] = 2

df2.loc[(df2.Dispozice == "Ostatní"), "Dispozice"] = 0.25
df2.loc[(df2.Dispozice == "Garsoniéra"), "Dispozice"] = 1
df2.loc[(df2.Dispozice == "1+kk"),    "Dispozice"] = 1.5
df2.loc[(df2.Dispozice == "1+1"),     "Dispozice"] = 1
df2.loc[(df2.Dispozice == "2+1"),     "Dispozice"] = 2
df2.loc[(df2.Dispozice == "2+kk"),    "Dispozice"] = 2.5
df2.loc[(df2.Dispozice == "3+1"),     "Dispozice"] = 3
df2.loc[(df2.Dispozice == "3+kk"),    "Dispozice"] = 3.5
df2.loc[(df2.Dispozice == "4+1"),     "Dispozice"] = 4
df2.loc[(df2.Dispozice == "4+kk"),    "Dispozice"] = 4.5
df2.loc[(df2.Dispozice == "5+1"),     "Dispozice"] = 5
df2.loc[(df2.Dispozice == "5+kk"),    "Dispozice"] = 5.5
df2.loc[(df2.Dispozice == "6+kk"),    "Dispozice"] = 6.5
df2.loc[(df2.Dispozice == "6+1"),    "Dispozice"] = 6
df2.loc[(df2.Dispozice == "7+kk"),    "Dispozice"] = 7.5
df2.loc[(df2.Dispozice == "7+1"),    "Dispozice"] = 7

#df2["Praha_sklep"] = df2["Praha"]*df2["Sklep"]
#df2["Praha_lodzie"] = df2["Praha"]*df2["Lodzie"]
#df2["Praha_parkovani"] = df2["Praha"]*df2["Parkovani"]

df2

/tmp/ipykernel_12835/1967476522.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Populace"][i] = mesta.get(df.iloc[:,5][i])
/tmp/ipykernel_12835/1967476522.py:67: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['Cena'] = df2['Cena'].str.replace('.','')


,Dispozice,Stav,Plocha,Cena,Mesto,Osobni_vl,PENB,Vybavenost,Balkon,Terasa,Sklep,Lodzie,Parkovani,Vytah,Garaz,Populace
0,2.5,3,38,19000000,Praha,1,7,2,0,0,0,1,0,1,0,1309000
1,2,1,63,3800000,Liberec,1,1,1,0,0,1,0,0,1,0,104163
2,1,2,27,2521000,Plzeň,1,1,1,0,0,1,0,0,0,0,171707
3,2,3,65,2290000,Ústí nad Labem,1,4,1,0,0,0,1,0,0,0,93020
4,4.5,4,112,5999000,Praha,0,6,1,0,0,1,1,0,1,1,1309000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,3,2,77,2695000,Jeseník,1,4,1,1,0,1,0,0,0,0,11204
799,3.5,3,73,7550000,Praha,1,3,1,0,0,1,0,1,0,0,1309000
800,3.5,5,91,13980000,Praha,1,4,2,0,0,1,0,0,1,0,1309000
801,2,2,59,6500000,Praha,1,4,2,0,0,1,1,0,1,0,1309000


In [210]:
train, test = train_test_split(df2, test_size=0.2)
train.head()

from sklearn.metrics import accuracy_score

Xtrain = train[["Dispozice", "Stav", "Plocha", "Populace", "Osobni_vl",  "PENB",
         "Vybavenost", "Balkon", "Terasa", "Lodzie", "Parkovani", "Sklep", "Garaz"]]# , "Lodzie", "Praha_lodzie", "Praha_parkovani", "Parkovani", "Praha_sklep", "Parkovani", "Sklep"
ytrain = train[["Cena"]]

Xtest = test[["Dispozice", "Stav", "Plocha", "Populace", "Osobni_vl",  "PENB",
         "Vybavenost", "Balkon", "Terasa", "Lodzie", "Parkovani", "Sklep", "Garaz"]]#  "
ytest = test[["Cena"]]

regr = LinearRegression()
regr.fit(Xtrain, ytrain)

print("Intercept: ", regr.intercept_)
print("Coefficients: ", regr.coef_)
print("R2 train = ", regr.score(Xtrain,ytrain)) #R2 train 0.71, R2 test 0.76, not bad
print("R2 test = ", regr.score(Xtest,ytest))

#test_byt = [3, 3, 79, 1, 1, 1, 3, 0, 0, 1, 1, 0, 0]
#regr.predict([test_byt])


Intercept:  [-3527888.53691294]
Coefficients:  [[ 2.63254820e+05  6.03665731e+04  7.33476948e+04  3.14114024e+00
   1.38511693e+06  1.00315129e+05  1.27460225e+05  9.98464398e+04
   5.27647868e+05 -4.64609782e+05 -2.38608021e+05 -1.72487242e+05
   8.66723758e+05]]
R2 train =  0.6967298958254139
R2 test =  0.6866260805159777


In [218]:
from sklearn.tree import DecisionTreeClassifier

forest = DecisionTreeClassifier(max_depth = 12)
forest.fit(Xtrain,ytrain)

print("accuracy train = ", forest.score(Xtrain,ytrain)) #overfit...
print("accuracy test = ", forest.score(Xtest,ytest))


accuracy train =  0.7987519500780031
accuracy test =  0.018633540372670808


In [9]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(Xtrain,ytrain)

print("accuracy train = ", knn.score(Xtrain,ytrain)) 
print("accuracy test = ", knn.score(Xtest,ytest))


accuracy train =  0.3
accuracy test =  0.0


/home/kub/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/kub/.local/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/kub/.local/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [88]:
df['Cena'] = df['Cena'].str.replace('.','')
df["Cena"] = df['Cena'].str[:-2]

df["Cena"] = pd.to_numeric(df["Cena"])
df.dropna(inplace=True)
df["Cena"] = df['Cena'].astype(int)

df_1pkk = df.loc[df["Dispozice"] == "1+kk"]
df_1p1 = df.loc[df["Dispozice"] == "1+1"]
df_2pkk = df.loc[df["Dispozice"] == "2+kk"]
df_2p1 = df.loc[df["Dispozice"] == "2+1"]
df_3pkk = df.loc[df["Dispozice"] == "3+kk"]
df_3p1 = df.loc[df["Dispozice"] == "3+1"]
df_gars = df.loc[df["Dispozice"] == "Garsoniéra"]

/tmp/ipykernel_12835/592047581.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Cena'] = df['Cena'].str.replace('.','')


In [89]:
print("1+kk", df_1pkk["Cena"].mean())
print("1+1",df_1p1["Cena"].mean())
print("2+kk",df_2pkk["Cena"].mean())
print("2+1",df_2p1["Cena"].mean())
print("3+kk",df_3pkk["Cena"].mean())
print("3+1",df_3p1["Cena"].mean())
print("ostatni", df_ostatni["Cena"].mean())
print("gars", df_gars["Cena"].mean())

1+kk 4329722.343137255
1+1 3295306.8771929825
2+kk 6005842.9147727275
2+1 4508140.25
3+kk 8605580.622222222
3+1 5179354.696969697
ostatni 953250.0
gars 2984000.0


{'Aš',
 'Beroun',
 'Besednice',
 'Borotín',
 'Brandýs nad Orlicí',
 'Brno',
 'Cerhenice',
 'Cheb',
 'Chlumec',
 'Chomutov',
 'Chotětov',
 'Chrudim',
 'Chvaletice',
 'Chýně',
 'Doksy',
 'Děčín',
 'Františkovy Lázně',
 'Frenštát pod Radhoštěm',
 'Frýdek-Místek',
 'Havířov',
 'Heřmanův Městec',
 'Holubice',
 'Horní Slavkov',
 'Horušice',
 'Hostivice',
 'Hořovice',
 'Hradec Králové',
 'Hradištko',
 'Hranice',
 'Hulín',
 'Hustopeče',
 'Jablonec nad Nisou',
 'Janov nad Nisou',
 'Jaroměř',
 'Jesenice',
 'Jeseník',
 'Jihlava',
 'Jinočany',
 'Jičín',
 'Kadaň',
 'Kamenice',
 'Karlovy Vary',
 'Karviná',
 'Kařez',
 'Kladno',
 'Klecany',
 'Klášterec nad Ohří',
 'Kolín',
 'Kopřivnice',
 'Kostelec nad Labem',
 'Kořenov',
 'Krupka',
 'Kutná Hora',
 'Kuřim',
 'Libavské Údolí',
 'Liberec',
 'Litvínov',
 'Loket',
 'Louny',
 'Lysá nad Labem',
 'Mariánské Lázně',
 'Mikulášovice',
 'Milešovice',
 'Milovice',
 'Mladá Boleslav',
 'Mníšek pod Brdy',
 'Mokrá-Horákov',
 'Moravský Beroun',
 'Most',
 'Měcholupy',


In [195]:
df2 = df
df2["Populace"] = 0
i = 0
for a in df2["Populace"]:
    df2["Populace"][i] = mesta.get(df2.iloc[:,5][i])
    #print(a)
    i=i+1
df2

/tmp/ipykernel_12835/1302624084.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Populace"][i] = mesta.get(df2.iloc[:,5][i])


,Cislo_inzeratu,Dispozice,Stav,Plocha,Cena,Mesto,Mestska,Vlastnictvi,Typ,PENB,...,Lodzie,Parkovani,Vytah,Garaz,K,Rekonstrukce,Odkaz,Datum_pridani,Datum_odebrani,Populace
0,677693,2+kk,Velmi dobrý,38 m²,19.000.000 Kč,Praha,Staré Město,Osobní,Ostatní,A,...,Ano,Ne,Ano,Ne,15. 09. 2021,vše,/home/kub/projects/bezrealitky/fotky/1,15:05:51,,1309000
1,691463,2+1,Špatný,63 m²,3.800.000 Kč,Liberec,-,Osobní,Panel,-,...,Ne,Ne,Ano,Ne,-,-,/home/kub/projects/bezrealitky/fotky/2,15:05:51,,104163
2,691460,1+1,Dobrý,27 m²,2.521.000 Kč,Plzeň,Doubravka,Osobní,Cihla,-,...,Ne,Ne,Ne,Ne,-,-,/home/kub/projects/bezrealitky/fotky/3,15:05:51,,171707
3,649809,2+1,Velmi dobrý,65 m²,2.290.000 Kč,Ústí nad Labem,-,Osobní,Panel,D,...,Ano,Ne,Ne,Ne,07. 02. 2021,-,/home/kub/projects/bezrealitky/fotky/4,15:05:51,,93020
4,681552,4+kk,Ve výstavbě,112 m²,5.999.000 Kč,Praha,Hlubočepy,Družstevní,Ostatní,B,...,Ano,Ne,Ano,Ano,01. 06. 2023,-,/home/kub/projects/bezrealitky/fotky/5,15:05:51,,1309000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,418519,3+1,Dobrý,77 m²,2.695.000 Kč,Jeseník,-,Osobní,Panel,D,...,Ne,Ne,Ne,Ne,01. 01. 1970,exteriér,/home/kub/projects/bezrealitky/fotky/799,15:07:33,,11204
799,632086,3+kk,Velmi dobrý,73 m²,7.550.000 Kč,Praha,Krč,Osobní,Cihla,E,...,Ne,Ano,Ne,Ne,01. 09. 2021,interiér,/home/kub/projects/bezrealitky/fotky/6,15:24:49,,1309000
800,691472,3+kk,Novostavba,91 m²,13.980.000 Kč,Praha,Bubeneč,Osobní,Cihla,D,...,Ne,Ne,Ano,Ne,01. 01. 2022,-,/home/kub/projects/bezrealitky/fotky/9,15:24:49,,1309000
801,690771,2+1,Dobrý,59 m²,6.500.000 Kč,Praha,Chodov,Osobní,Panel,D,...,Ano,Ne,Ano,Ne,15. 11. 2021,-,/home/kub/projects/bezrealitky/fotky/10,15:24:49,,1309000


In [199]:
df2

,Dispozice,Stav,Plocha,Cena,Mesto,Osobni_vl,PENB,Vybavenost,Balkon,Terasa,Sklep,Lodzie,Parkovani,Vytah,Garaz,Populace
0,2+kk,Velmi dobrý,38 m²,19.000.000 Kč,Praha,1,A,Částečně,Ne,Ne,Ne,Ano,Ne,Ano,Ne,NaN
1,2+1,Špatný,63 m²,3.800.000 Kč,Liberec,1,-,-,Ne,Ne,Ano,Ne,Ne,Ano,Ne,NaN
2,1+1,Dobrý,27 m²,2.521.000 Kč,Plzeň,1,-,-,Ne,Ne,Ano,Ne,Ne,Ne,Ne,NaN
3,2+1,Velmi dobrý,65 m²,2.290.000 Kč,Ústí nad Labem,1,D,Nevybavený,Ne,Ne,Ne,Ano,Ne,Ne,Ne,NaN
4,4+kk,Ve výstavbě,112 m²,5.999.000 Kč,Praha,0,B,Nevybavený,Ne,Ne,Ano,Ano,Ne,Ano,Ano,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,3+1,Dobrý,77 m²,2.695.000 Kč,Jeseník,1,D,Nevybavený,Ano,Ne,Ano,Ne,Ne,Ne,Ne,NaN
799,3+kk,Velmi dobrý,73 m²,7.550.000 Kč,Praha,1,E,Nevybavený,Ne,Ne,Ano,Ne,Ano,Ne,Ne,NaN
800,3+kk,Novostavba,91 m²,13.980.000 Kč,Praha,1,D,Částečně,Ne,Ne,Ano,Ne,Ne,Ano,Ne,NaN
801,2+1,Dobrý,59 m²,6.500.000 Kč,Praha,1,D,Částečně,Ne,Ne,Ano,Ano,Ne,Ano,Ne,NaN


In [220]:
df2.Cena.describe()

count    8.020000e+02
mean     6.239024e+06
std      4.272603e+06
min      5.990000e+02
25%      3.399250e+06
50%      5.450000e+06
75%      7.599000e+06
max      4.491000e+07
Name: Cena, dtype: float64

In [221]:
df2["Cena"].describe()

count    8.020000e+02
mean     6.239024e+06
std      4.272603e+06
min      5.990000e+02
25%      3.399250e+06
50%      5.450000e+06
75%      7.599000e+06
max      4.491000e+07
Name: Cena, dtype: float64